In [1]:
import polars as pl
from cleaner import sanitize
from geopy import distance
import math
import pydeck

In [2]:
df = pl.read_parquet("data/processed.parquet")

In [3]:
df.shape

(2125763, 32)

In [4]:
df.head()

borough,zip_code,latitude,longitude,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,date,year,month,time,hour,number_of_casualty,weekday
str,i64,f64,f64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,date,i32,i8,time,i8,i64,i8
null,null,null,null,"""WHITESTONE EXPRESSWAY""","""20 AVENUE""",null,2,0,0,0,0,0,2,0,"""Aggressive Driving/Road Rage""","""Unspecified""",null,null,null,"""Sedan""","""Sedan""",null,null,null,2021-09-11,2021,9,02:39:00,2,2,6
null,null,null,null,"""QUEENSBORO BRIDGE UPPER""",null,null,1,0,0,0,0,0,1,0,"""Pavement Slippery""",null,null,null,null,"""Sedan""",null,null,null,null,2022-03-26,2022,3,11:45:00,11,1,6
null,null,null,null,"""THROGS NECK BRIDGE""",null,null,0,0,0,0,0,0,0,0,"""Following Too Closely""","""Unspecified""",null,null,null,"""Sedan""","""Pick-up Truck""",null,null,null,2022-06-29,2022,6,06:55:00,6,0,3
"""BROOKLYN""",11208,40.667202,-73.8665,null,null,"""1211 LORING AVENUE""",0,0,0,0,0,0,0,0,"""Unspecified""",null,null,null,null,"""Sedan""",null,null,null,null,2021-09-11,2021,9,09:35:00,9,0,6
"""BROOKLYN""",11233,40.683304,-73.917274,"""SARATOGA AVENUE""","""DECATUR STREET""",null,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,2021-12-14,2021,12,08:13:00,8,0,2


In [5]:
df = df.with_columns(pl.col("date").dt.weekday().alias("weekday"))
df.head()

borough,zip_code,latitude,longitude,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,date,year,month,time,hour,number_of_casualty,weekday
str,i64,f64,f64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,date,i32,i8,time,i8,i64,i8
null,null,null,null,"""WHITESTONE EXPRESSWAY""","""20 AVENUE""",null,2,0,0,0,0,0,2,0,"""Aggressive Driving/Road Rage""","""Unspecified""",null,null,null,"""Sedan""","""Sedan""",null,null,null,2021-09-11,2021,9,02:39:00,2,2,6
null,null,null,null,"""QUEENSBORO BRIDGE UPPER""",null,null,1,0,0,0,0,0,1,0,"""Pavement Slippery""",null,null,null,null,"""Sedan""",null,null,null,null,2022-03-26,2022,3,11:45:00,11,1,6
null,null,null,null,"""THROGS NECK BRIDGE""",null,null,0,0,0,0,0,0,0,0,"""Following Too Closely""","""Unspecified""",null,null,null,"""Sedan""","""Pick-up Truck""",null,null,null,2022-06-29,2022,6,06:55:00,6,0,3
"""BROOKLYN""",11208,40.667202,-73.8665,null,null,"""1211 LORING AVENUE""",0,0,0,0,0,0,0,0,"""Unspecified""",null,null,null,null,"""Sedan""",null,null,null,null,2021-09-11,2021,9,09:35:00,9,0,6
"""BROOKLYN""",11233,40.683304,-73.917274,"""SARATOGA AVENUE""","""DECATUR STREET""",null,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,2021-12-14,2021,12,08:13:00,8,0,2


In [6]:
# df = df.with_columns(pl.col("crash_date").str.to_date("%m/%d/%Y").alias("date"))
# df = df.with_columns(pl.col("date").dt.year().alias("year"))
# df = df.with_columns(pl.col("date").dt.month().alias("month"))
# df = df.with_columns(pl.col("crash_time").str.to_time("%H:%M").alias("time"))
# df = df.with_columns(pl.col("time").dt.hour().alias("hour"))

# # df[["date", "year", "month", "time", "hour"]].head()

In [7]:
# df = df.drop(["crash_date", "crash_time", "location"])
# df = df.drop('collision_id')
df.write_parquet('data/processed.parquet', use_pyarrow=True)

In [8]:
# df[["date", "borough", "year", "month", "time", "hour"]].write_parquet("data/time_only.parquet", use_pyarrow=True)

In [9]:
clean = pl.read_parquet("data/clean_data.parquet")

In [10]:
clean.head()

borough,zip_code,latitude,longitude,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,date,year,month,time,hour,number_of_casualty,distance
str,i64,f64,f64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,date,i32,i8,time,i8,i64,f64
"""BROOKLYN""",11208,40.667202,-73.8665,null,null,"""1211 LORING AVENUE""",0,0,0,0,0,0,0,0,"""Unspecified""",null,null,null,null,"""Sedan""",null,null,null,null,2021-09-11,2021,9,09:35:00,9,0,12.832897
"""BROOKLYN""",11233,40.683304,-73.917274,"""SARATOGA AVENUE""","""DECATUR STREET""",null,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,2021-12-14,2021,12,08:13:00,8,0,8.183083
"""BRONX""",10475,40.86816,-73.83148,null,null,"""344 BAYCHESTER AVENUE""",2,0,0,0,0,0,2,0,"""Unspecified""","""Unspecified""",null,null,null,"""Sedan""","""Sedan""",null,null,null,2021-12-14,2021,12,08:17:00,8,2,22.685206
"""BROOKLYN""",11207,40.67172,-73.8971,null,null,"""2047 PITKIN AVENUE""",0,0,0,0,0,0,0,0,"""Driver Inexperience""","""Unspecified""",null,null,null,"""Sedan""",null,null,null,null,2021-12-14,2021,12,21:10:00,21,0,10.273211
"""MANHATTAN""",10017,40.75144,-73.97397,"""3 AVENUE""","""EAST 43 STREET""",null,0,0,0,0,0,0,0,0,"""Passing Too Closely""","""Unspecified""",null,null,null,"""Sedan""","""Station Wagon/Sport Utility Ve…",null,null,null,2021-12-14,2021,12,14:58:00,14,0,5.07277


In [11]:
# clean = df.drop_nulls(subset=["latitude"])
# clean = clean.drop_nulls(subset=["longitude"])
# clean = clean.drop_nulls(subset=["borough"])

In [12]:
# clean.shape

In [13]:
clean = clean.with_columns((pl.concat_list(pl.col("latitude"), pl.col("longitude")).alias("coordinate")))

In [14]:
clean = clean.with_columns(pl.lit((40.7128, -74.0060)).alias('ny'))
# clean.head()

In [15]:
# clean = clean.with_columns((pl.concat_list(pl.col("coordinate"), pl.col("ny")).alias("geodesic")))
# clean.head()

In [16]:
# clean['geodesic'].dtype

In [17]:
# clean = clean.with_columns(pl.col("geodesic").map_elements(lambda x: distance.geodesic((x[0], x[1]), (x[2], x[3])).km, return_dtype=float).alias("distance"))
# clean.head()

In [18]:
# clean = clean.drop(["geodesic", "ny", "coordinate"])

In [19]:
clean.shape

(1411081, 34)

In [20]:
# clean.filter(pl.col('distance') > 530).write_parquet("data/invalid_coordinate.parquet")
# clean = clean.filter(pl.col("distance") < threshold)

In [21]:
# threshold = 3*clean['distance'].mean()
# clean = clean.filter(pl.col("distance") < threshold)
# clean.shape

In [22]:
colors = [
    '#0044ff',  # muted blue
    '#ffaa00',  # muted purple
    '#00ff00',  # chestnut brown
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    
]
boroughs = clean['borough'].unique()
colors = colors[:boroughs.shape[0]]

In [23]:
clean.head()

borough,zip_code,latitude,longitude,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,date,year,month,time,hour,number_of_casualty,distance,coordinate,ny
str,i64,f64,f64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,date,i32,i8,time,i8,i64,f64,list[f64],list[f64]
"""BROOKLYN""",11208,40.667202,-73.8665,null,null,"""1211 LORING AVENUE""",0,0,0,0,0,0,0,0,"""Unspecified""",null,null,null,null,"""Sedan""",null,null,null,null,2021-09-11,2021,9,09:35:00,9,0,12.832897,"[40.667202, -73.8665]","[40.7128, -74.006]"
"""BROOKLYN""",11233,40.683304,-73.917274,"""SARATOGA AVENUE""","""DECATUR STREET""",null,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,2021-12-14,2021,12,08:13:00,8,0,8.183083,"[40.683304, -73.917274]","[40.7128, -74.006]"
"""BRONX""",10475,40.86816,-73.83148,null,null,"""344 BAYCHESTER AVENUE""",2,0,0,0,0,0,2,0,"""Unspecified""","""Unspecified""",null,null,null,"""Sedan""","""Sedan""",null,null,null,2021-12-14,2021,12,08:17:00,8,2,22.685206,"[40.86816, -73.83148]","[40.7128, -74.006]"
"""BROOKLYN""",11207,40.67172,-73.8971,null,null,"""2047 PITKIN AVENUE""",0,0,0,0,0,0,0,0,"""Driver Inexperience""","""Unspecified""",null,null,null,"""Sedan""",null,null,null,null,2021-12-14,2021,12,21:10:00,21,0,10.273211,"[40.67172, -73.8971]","[40.7128, -74.006]"
"""MANHATTAN""",10017,40.75144,-73.97397,"""3 AVENUE""","""EAST 43 STREET""",null,0,0,0,0,0,0,0,0,"""Passing Too Closely""","""Unspecified""",null,null,null,"""Sedan""","""Station Wagon/Sport Utility Ve…",null,null,null,2021-12-14,2021,12,14:58:00,14,0,5.07277,"[40.75144, -73.97397]","[40.7128, -74.006]"


In [24]:
# color_lookup = pydeck.data_utils.assign_random_colors(clean['borough'])
# colors = list(color_lookup.values())

In [25]:
# clean = clean.with_columns(pl.col("borough").map_elements(lambda x: color_lookup[x], return_dtype=list[int]).alias("color"))
# clean = clean.with_columns(pl.col("borough").str.replace_many(boroughs, colors).alias("color"))

In [26]:
clean.write_parquet("data/clean_data.parquet", use_pyarrow=True)

In [27]:
map_cols = ["date", "borough", "latitude", "longitude", "month", "year", "time", "hour", "coordinate", "number_of_persons_injured", "number_of_persons_killed", "number_of_casualty", "distance"]
clean.select(map_cols).write_parquet("data/clean_map.parquet", use_pyarrow=True)